In [1]:
import numpy as np
import utils
import pandas as pd

In [2]:
synth_regular = utils.load_synthetic()
compas = utils.load_compas_alt()
RESPONSE = "score_factor"

In [3]:
compas.keys()

dict_keys(['train', 'test'])

In [4]:
#compas["standard"]["train"].keys()

In [5]:
#Checking that the classes don't become too much more imbalanced when data is removed.

#Synthetic data
for key, value in synth_regular.items():
    #print(value.columns)
    print("Key: ", key)
    print("Caucasian = 1: ",len(value[value["is_Caucasian"]==1])/len(value) ,
    "\nCaucasian = 0: ", len(value[value["is_Caucasian"]==0])/len(value))
    print("Male = 1: ",len(value[value["gender_factor"]==1])/len(value) ,
    "\nMale = 0: ", len(value[value["gender_factor"]==0])/len(value))
    print("After 30% missing: ")
    temp = value.copy()
    temp = utils.impute(utils.data_remover_cat(temp, "is_Caucasian", 30), "is_Caucasian")
    print("Caucasian = 1: ",len(temp[temp["is_Caucasian"]==1])/len(temp) ,
        "\nCaucasian = 0: ", len(temp[temp["is_Caucasian"]==0])/len(temp))
    print("Male = 1: ",len(temp[temp["gender_factor"]==1])/len(temp) ,
        "\nMale = 0: ", len(temp[temp["gender_factor"]==0])/len(temp))

Key:  test
Caucasian = 1:  0.34634634634634637 
Caucasian = 0:  0.6536536536536537
Male = 1:  0.8043043043043043 
Male = 0:  0.1956956956956957
After 30% missing: 
Caucasian = 1:  0.3048128342245989 
Caucasian = 0:  0.6951871657754011
Male = 1:  0.8349885408708938 
Male = 0:  0.1650114591291062
Key:  train
Caucasian = 1:  0.34007996001999 
Caucasian = 0:  0.65992003998001
Male = 1:  0.809095452273863 
Male = 0:  0.19090454772613694
After 30% missing: 
Caucasian = 1:  0.2875375375375375 
Caucasian = 0:  0.7124624624624625
Male = 1:  0.8490990990990991 
Male = 0:  0.15090090090090091


In [6]:
a = []
b = [1,2,3]
c = [4,5,56]
a.append(b)
a.append(c)
np.mean(a, axis = 0)

array([ 2.5,  3.5, 29.5])

In [7]:
#Compas data
"""for ver in ["standard", "violent"]:
    print("Compas dataset type: ", ver)"""
for key, value in compas.items():
    #print(value.columns)
    print("Key: ", key)
    print("Caucasian = 1: ",len(value[value["is_Caucasian"]==1])/len(value) ,
    "\nCaucasian = 0: ", len(value[value["is_Caucasian"]==0])/len(value))
    print("Male = 1: ",len(value[value["gender_factor"]==1])/len(value) ,
    "\nMale = 0: ", len(value[value["gender_factor"]==0])/len(value))
    print("Score text: ", len(value[value["score_factor"]==1])/len(value))
    print("After 30% missing: ")
    temp = value.copy()
    temp = utils.impute(utils.data_remover_cat(temp, "is_Caucasian", 30), "is_Caucasian")
    print("Caucasian = 1: ",len(temp[temp["is_Caucasian"]==1])/len(temp) ,
        "\nCaucasian = 0: ", len(temp[temp["is_Caucasian"]==0])/len(temp))
    print("Male = 1: ",len(temp[temp["gender_factor"]==1])/len(temp) ,
        "\nMale = 0: ", len(temp[temp["gender_factor"]==0])/len(temp))
    print("Score text: ", len(value[value["score_factor"]==1])/len(value))


Key:  train
Caucasian = 1:  0.3388149939540508 
Caucasian = 0:  0.6611850060459492
Male = 1:  0.8099153567110037 
Male = 0:  0.19008464328899638
Score text:  0.44401451027811367
After 30% missing: 
Caucasian = 1:  0.32952924393723254 
Caucasian = 0:  0.6704707560627675
Male = 1:  0.8088445078459344 
Male = 0:  0.19115549215406563
Score text:  0.44401451027811367
Key:  test
Caucasian = 1:  0.3446244477172312 
Caucasian = 0:  0.6553755522827688
Male = 1:  0.8090328915071183 
Male = 0:  0.19096710849288168
Score text:  0.4491899852724595
After 30% missing: 
Caucasian = 1:  0.3361462728551336 
Caucasian = 0:  0.6638537271448663
Male = 1:  0.7883263009845288 
Male = 0:  0.21167369901547117
Score text:  0.4491899852724595


In [8]:
percentiles = [p for p in range(0,20,2)]+[p for p in range(20,60, 10)]
missing=["priors_count"]#,"two_year_recid", "is_Caucasian", "gender_factor", "crime_factor"]
all_results = {}
for miss in missing:
    for sensitive in ["is_Caucasian"]:#, "gender_factor"]:
        synth_results = utils.test_bench(train = synth_regular["train"],test = synth_regular["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                            percentiles = percentiles)
        recid_results = utils.test_bench(train = compas["train"],test = compas["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                        percentiles = percentiles)
        #TODO and remember to fix data v_recid_results = utils.test_bench(train = compas["standard"]["train"],test = compas["standard"]["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                        #percentiles = percentiles)
        all_results[miss+"_"+sensitive+"_"+"synth"] = synth_results 
        all_results[miss+"_"+sensitive+"_"+"recid"] = recid_results 
        """try:
            synth_results = utils.test_bench(train = synth_regular["train"],test = synth_regular["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                            percentiles = percentiles)
            recid_results = utils.test_bench(train = compas["train"],test = compas["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                            percentiles = percentiles)
            #TODO and remember to fix data v_recid_results = utils.test_bench(train = compas["standard"]["train"],test = compas["standard"]["test"], pred = RESPONSE, missing = miss, sensitive=sensitive,
                            #percentiles = percentiles)
            all_results[miss+"_"+sensitive+"_"+"synth"] = synth_results 
            all_results[miss+"_"+sensitive+"_"+"recid"] = recid_results 
            #all_results[miss+"_"+sensitive+"_"+"v_recid"] = v_recid_results
        except Exception as e:
            print("Exception: ", e)
            print("Parameters: ", sensitive+"_"+miss) 
        """

100%|██████████| 4/4 [01:32<00:00, 23.19s/it]


In [9]:
import json
from pathlib import Path
with open(Path("raw_data/second.json"), 'w') as f:
            json.dump(all_results, f)

In [10]:
for key, res in all_results.items():
    cm_df = utils.plotting_cf(["log_reg", "rf_cat", "svm", "knn"],["cca", "mice_def", "mean"], res, key+"/")
    metric_df = utils.plotting_others(res, key+"/")

findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.
findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.
findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.


{'log_reg_cca_mar': {'pp': 0.0}, 'log_reg_mean_mar': {'pp': 0.0}, 'log_reg_mice_reg_mar': {}, 'log_reg_mice_def_mar': {'pp': 0.0}, 'log_reg_reg_mar': {}, 'rf_cat_cca_mar': {'pp': 0.21335504885993484}, 'rf_cat_mean_mar': {'pp': 0.05294117647058827}, 'rf_cat_mice_reg_mar': {}, 'rf_cat_mice_def_mar': {'pp': 0.006172839506172867}, 'rf_cat_reg_mar': {}, 'svm_cca_mar': {'pp': 0.0}, 'svm_mean_mar': {'pp': 0.0}, 'svm_mice_reg_mar': {}, 'svm_mice_def_mar': {'pp': 0.0}, 'svm_reg_mar': {}, 'knn_cca_mar': {'pp': 0.026545932977323772}, 'knn_mean_mar': {'pp': 0.0}, 'knn_mice_reg_mar': {}, 'knn_mice_def_mar': {'pp': 0.0}, 'knn_reg_mar': {}, 'log_reg_cca_mcar': {'pp': 0.0}, 'log_reg_mean_mcar': {'pp': 0.0}, 'log_reg_mice_reg_mcar': {}, 'log_reg_mice_def_mcar': {'pp': 0.0}, 'log_reg_reg_mcar': {}, 'rf_cat_cca_mcar': {'pp': 0.0}, 'rf_cat_mean_mcar': {'pp': 0.0}, 'rf_cat_mice_reg_mcar': {}, 'rf_cat_mice_def_mcar': {'pp': 0.012269938650306789}, 'rf_cat_reg_mcar': {}, 'svm_cca_mcar': {'pp': 0.0}, 'svm_mean

<Figure size 1332x756 with 0 Axes>

In [11]:
print(cm_df)

         log_reg_cca_MCAR  log_reg_cca_MAR  log_reg_mice_def_MCAR  \
TPR Z=1          0.491597         0.600840               0.542017   
TPR Z=0          0.760709         0.646972               0.741507   
TNR Z=1          0.922414         0.834052               0.898707   
TNR Z=0          0.705167         0.755319               0.740122   

         log_reg_mice_def_MAR  log_reg_mean_MCAR  log_reg_mean_MAR  \
TPR Z=1              0.617647           0.466387          0.462185   
TPR Z=0              0.772526           0.742984          0.744461   
TNR Z=1              0.838362           0.931034          0.922414   
TNR Z=0              0.697568           0.735562          0.702128   

         rf_cat_cca_MCAR  rf_cat_cca_MAR  rf_cat_mice_def_MCAR  \
TPR Z=1         0.508403        0.529412              0.487395   
TPR Z=0         0.722304        0.627770              0.676514   
TNR Z=1         0.913793        0.887931              0.905172   
TNR Z=0         0.744681        0.77963

In [12]:
print(metric_df)

        log_reg_cca_mar  log_reg_mean_mar  log_reg_mice_reg_mar  \
spd                 NaN               NaN                   NaN   
pp             0.081219          0.033425                   NaN   
eo_Y=0              NaN               NaN                   NaN   
eo_Y=1              NaN               NaN                   NaN   

        log_reg_mice_def_mar  log_reg_reg_mar  rf_cat_cca_mar  \
spd                      NaN              NaN             NaN   
pp                  0.062215              NaN        0.037749   
eo_Y=0                   NaN              NaN             NaN   
eo_Y=1                   NaN              NaN             NaN   

        rf_cat_mean_mar  rf_cat_mice_reg_mar  rf_cat_mice_def_mar  \
spd                 NaN                  NaN                  NaN   
pp             0.016307                  NaN             0.067356   
eo_Y=0              NaN                  NaN                  NaN   
eo_Y=1              NaN                  NaN                  